In [1]:
import psycopg2
from sqlalchemy import create_engine, text
import json
import pandas as pd
try:
    from urllib import parse, request
except:
    from urlparse import urlparse as parse
import urllib
import urllib2
from json import loads
import simplejson
import pysolr
from textblob import TextBlob as tb
import requests

host="132.249.238.27"
dbname='bookstore_dp'
user='student'
password='123456'

pg_connstring='postgresql+psycopg2://{0}:{1}@{2}:5432/{3}'.format(user, password, host, dbname)
astx_host="132.249.238.32" # not open or not running in ucsd
solr_host="132.249.238.28" # not open or not running in ucsd

In [77]:
def Sales_Reviews(category, month):
    # AsterixDBConnection
    class QueryResponse:
        def __init__(self, raw_response):
            self._json = loads(raw_response)

            self.requestID = self._json['requestID'] if 'requestID' in self._json else None
            self.clientContextID = self._json['clientContextID'] if 'clientContextID' in self._json else None
            self.signature = self._json['signature'] if 'signature' in self._json else None
            self.results = self._json['results'] if 'results' in self._json else None
            self.metrics = self._json['metrics'] if 'metrics' in self._json else None

    class AsterixConnection:
        def __init__(self, server='http://{0}'.format(astx_host), port=19002):

            self._server = server
            self._port = port
            self._url_base = self._server + ':' + str(port)

            # print ("connecting to: " + server)

        def query(self, statement, pretty=False, client_context_id=None):
            endpoint = '/query/service'

            url = self._url_base + endpoint

            payload = {
                'statement': statement,
                'pretty': pretty
            }

            if client_context_id:
                payload['client_context_id'] = client_context_id
            try:
                data = parse.urlencode(payload).encode("utf-8")
                req = request.Request(url, data)
                response = request.urlopen(req).read()
            except:
                data = urllib.urlencode(payload)
                data = data.encode("utf-8")
                req = urllib2.Request(url, data)
                response = urllib2.urlopen(req).read()
            # print(data)

            return QueryResponse(response)

    asterix_conn = AsterixConnection()
    axquery = '''
        use bookstore_dp;
        SELECT * from ClassificationInfo where classification LIKE "%{0}%";'''.format(category)

    response = asterix_conn.query(axquery)


    df = pd.DataFrame(response.results)
    node_id = []
    for i in range(df.shape[0]):
        a = df.ClassificationInfo[i]['nodeID']
        node_id.append(a)
    node_id = [str(x) for x in node_id]
    node_id = set(node_id)
    conn_string = "host='{0}' dbname='{1}' user='{2}' password='{3}'".format(host, dbname, user, password)
    #print ("Connecting to database\n	->%s" % (conn_string))
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    #print ("Connected!\n")

    sql = "SELECT DISTINCT o.productid, o.billdate, o.numunits, p.asin,p.nodeid\
        FROM orderlines o, products p\
        WHERE o.productid=p.productid\
        AND EXTRACT(month from billdate)={0};".format(month)

    cursor.execute(sql)

    # retrieve the records from the database
    records = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
    df1 = pd.DataFrame(records, columns=colnames)
    df2 = df1[df1['nodeid'].isin(node_id)]
    df3 = df2.groupby(['billdate', 'productid', 'asin', 'nodeid'], as_index=False)['numunits'].sum()
    #print df3
    my_asin = df3['asin']
    my_asin = set(my_asin)
    asin = ', '.join(my_asin)

    def solrWrap(core, params):
        query_string = 'http://{0}:8983/solr/{1}/select?'.format(solr_host, core)  # connecting to our linode server
        for key in params:
            query_string = query_string + key + '=' + params[key] + '&'

        # print(query_string)

        solrcon = pysolr.Solr(query_string, timeout=10)
        results = solrcon.search('*:*')
        docs = pd.DataFrame(results.docs)
        
        return docs

    d3 = {'q': 'asin:(%s)' % asin, 'rows': '77165'}
    
    d_res3 = solrWrap(dbname, d3)
    
    polarity_measure = []
    for i in range(d_res3.shape[0]):
        str1 = str(d_res3.reviewText[i])
        blob = tb(str1)
        polarity_measure.append(blob.sentiment.polarity)

    se = pd.Series(polarity_measure)
    
    d_res3['Sentiment_polarity'] = se.values
    
    d_res3['asin'] = d_res3['asin'].apply(lambda x: '' + str(x)[3:-2] + '')
   
    df_sentiment = d_res3.groupby(['asin'], as_index=False)['Sentiment_polarity'].mean()
    
    
    result = pd.merge(df3, df_sentiment, on='asin')
    
    final_result = result.reset_index().to_json(orient='records')

    return final_result

In [78]:
Sales_Reviews('Education',2)

'[{"index":0,"billdate":1296518400000,"productid":11083,"asin":"0005126207","nodeid":"7009083011","numunits":1,"Sentiment_polarity":0.1951358826},{"index":1,"billdate":1296604800000,"productid":11083,"asin":"0005126207","nodeid":"7009083011","numunits":1,"Sentiment_polarity":0.1951358826},{"index":2,"billdate":1296691200000,"productid":11083,"asin":"0005126207","nodeid":"7009083011","numunits":1,"Sentiment_polarity":0.1951358826},{"index":3,"billdate":1296777600000,"productid":11083,"asin":"0005126207","nodeid":"7009083011","numunits":3,"Sentiment_polarity":0.1951358826},{"index":4,"billdate":1297123200000,"productid":11083,"asin":"0005126207","nodeid":"7009083011","numunits":1,"Sentiment_polarity":0.1951358826},{"index":5,"billdate":1297209600000,"productid":11083,"asin":"0005126207","nodeid":"7009083011","numunits":1,"Sentiment_polarity":0.1951358826},{"index":6,"billdate":1297296000000,"productid":11083,"asin":"0005126207","nodeid":"7009083011","numunits":1,"Sentiment_polarity":0.19